In [1]:
import numpy as np 

In [2]:
from Enviar import Receber

In [3]:
import jscatter as js

In [4]:
from scipy import stats

In [5]:
import math 

In [6]:
import scipy.interpolate, scipy.optimize

In [7]:
receber=Receber(nomeRef='08-04-2020-teste-11')
receber()
dados=receber.dados   

In [8]:
concentracao =[0, 0.021, 0.025, 0.04, 0.05,  0.06, 0.07,  0.075, 0.08, 0.09,  0.1,  0.11, 0.12,0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.198, 0.2, 0.22]

In [19]:
class Resultado():
    def __init__(self, **kwargs):
        self.dados=kwargs.get('dados')
        
    def plotDiagrama(self, L=10, A=1.7, **kargs):
        concentracao=kargs.get("concentracao")
        p=js.grace()
        p.yaxis(label='tc',charsize=1.50)
        p.xaxis(label='q',charsize=1.50)
        x=[]
        y=[]   
        for q in concentracao: 
            for dado in self.dados:
                if dado.get('p')== q and dado.get('L')==L and dado.get('A')==A:
                    y.append(dado.get('tc'))
                    x.append(q)
        p.plot(x,y,symbol=-1,line=[1,1,''],legend='Q=$q')
    
   
    def plotDiagramaFiltrado(self,L=10, A=1.7, **kargs):
        concentracao=kargs.get("concentracao")
        p=js.grace()
        p.yaxis(label='tc',charsize=1.50)
        p.xaxis(label='q',charsize=1.50)
        for q in concentracao: 
            x=[]
            y=[]   
            for dado in self.dados:
                if dado.get('p')== q and dado.get('L')==L and dado.get('A')==A:
                    y.append(dado.get('tc'))
                    x.append(q)
            zscore=stats.zscore(np.array(y))
            X=[]
            Y=[]
            for xx,yy,zz in zip(x, y, zscore.tolist()):
                if abs(zz) < 2.5:
                    Y.append(yy)
                    X.append(xx)
            p.plot(X,Y,symbol=-1,line=[1,1,''],legend='Q=$q')

    def plotDiagramaMedio(self, L=10, A=1.7, **kargs):
        concentracao=kargs.get("concentracao")
        p=js.grace()
        p.yaxis(label='tc',charsize=1.50)
        p.xaxis(label='q',charsize=1.50)
        xmedia=[]
        ymedia=[]
        yErro=[]
        for q in concentracao: 
            x=[]
            y=[]   
            for dado in self.dados:
                if dado.get('p')== q and dado.get('L')==L and dado.get('A')==A:
                    y.append(dado.get('tc'))
                    x.append(q)
            zscore=stats.zscore(np.array(y))
            X=[]
            Y=[]
            for xx,yy,zz in zip(x, y, zscore.tolist()):
                if abs(zz) < 2.5:
                    Y.append(yy)
                    X.append(xx)
            ymedia.append(np.average(Y))
            yErro.append(np.std(Y)/(math.sqrt(len(Y))))
            xmedia.append(np.average(X))
        p.plot(xmedia,ymedia,yErro,symbol=-1,line=[1,1,''],legend='p')
   
    def plot(self,L=10, A=1.7, x='t', y='sus', **kargs):
        concentracao=kargs.get("concentracao")
        p=js.grace()
        p.yaxis(label=y,charsize=1.50)
        p.xaxis(label=x,charsize=1.50)
        for i, q in enumerate(concentracao):
            for dado in self.dados:
                if dado.get('p')== q and dado.get('L')==L and dado.get('A')==A:
                    p.plot(dado.get(x), dado.get(y), symbol=[0,0.3,i], line =[1,1,i])
                    
    
    def plotMedia(self,L=10, A=1.7, x='t', y='sus', **kargs):
        concentracao=kargs.get("concentracao")
        tamanhos=kargs.get("tamanhos")
        if not tamanhos:
            tamanhos=[L]
        p=js.grace()
        p.yaxis(label=y,charsize=1.50)
        p.xaxis(label=x,charsize=1.50)  
        cont=0
        for L in tamanhos:
            for i, q in enumerate(concentracao):
                somaY=np.zeros(len(dados[0].get(x)), dtype=float)
                cont=0
                for dado in self.dados:
                    if dado.get('p')== q and dado.get('L')==L and dado.get('A')==A:
                        somaY += np.array(dado.get(y))
                        if cont==0 :
                            xx=dado.get(x)
                        cont += 1
                somaY= somaY/cont
                p.plot(xx, somaY.tolist(), symbol=[0,0.3,i], line =[1,2,i] ,legend='q='+repr(q))

    def plotDiagramaCruzamentoMedia(self,L=10, A=1.7, x='t', y='sus', **kargs):
        concentracao=kargs.get("concentracao")
        tamanhos=kargs.get("tamanhos")
        if not tamanhos:
            tamanhos=[L]
        p=js.grace()
        p.yaxis(label=y,charsize=1.50)
        p.xaxis(label=x,charsize=1.50)  
        cont=0
        tcs=[]
        for i, q in enumerate(concentracao):
            somaY1=np.zeros(len(dados[0].get(x)), dtype=float)
            somaY2=np.zeros(len(dados[0].get(x)), dtype=float)
            cont1=0
            cont2=0
            for dado in self.dados:
                if dado.get('p')== q and dado.get('L')==tamanhos[0] and dado.get('A')==A:
                    somaY1 += np.array(dado.get(y))
                    if cont==0 :
                        xx1=dado.get(x)
                    cont1 += 1
                if dado.get('p')== q and dado.get('L')==tamanhos[1] and dado.get('A')==A:
                    somaY2 += np.array(dado.get(y))
                    if cont==0 :
                        xx2=dado.get(x)
                        t0=dado.get("tc")
                    cont2 += 1
            somaY1= somaY1/cont1
            somaY2= somaY2/cont2
            tc=self.cruzamento(xx1,somaY1.tolist(),xx2, somaY2.tolist(), t0)[0]
            tcs.append(tc)
        print(tcs)
        p.plot(concentracao, tcs, symbol=[0,0.3,1], line =[1,2,1] )


    def plotFiltrados(self, x='t', y='sus', L=10, A=1.7, **kargs):
        concentracao=kargs.get("concentracao")
        p=js.grace()
        p.yaxis(label=y,charsize=1.50)
        p.xaxis(label=x,charsize=1.50)
        for i, q in enumerate(concentracao):
            Y=[]
            CurvasX=[]
            CurvasY=[]
            for dado in self.dados:
                if dado.get('p')== q and dado.get('L')==L and dado.get('A')==A:
                    Y.append(dado.get('tc'))
                    CurvasX.append(dado.get(x))
                    CurvasY.append(dado.get(y))
            zscore=stats.zscore(np.array(Y))        
            for zz,tt,suss in zip(zscore.tolist(),CurvasX,CurvasY):
                if abs(zz) < 2.5:
                    p.plot(tt,suss, symbol=[0,0.3,i], line  =[1,1,i])#,legend='Q=$q')
    
    def PlotCruzamentosNormal(self,A=1.7, **kargs):
        concentracao=kargs.get("concentracao")
        tamanhos=kargs.get("tamanhos")
        tcmedio=[]
        tcerro=[]
        p=js.grace()
        p.yaxis(label='tc',charsize=1.50)
        p.xaxis(label='q',charsize=1.50)
        for q in concentracao:
            listasx1=[]
            listasy1=[]
            listast01=[]

            listasx2=[]
            listasy2=[]
            listast02=[]
            for dado in self.dados:
                if dado.get('p')== q and dado.get('L')==tamanhos[0] and dado.get('A')==A:
                    listasx1.append(dado.get('t'))
                    listasy1.append(dado.get('cumu'))
                    listast01.append(dado.get('tc'))
                if dado.get('p')== q and dado.get('L')==tamanhos[1] and dado.get('A')==A:
                    listasx2.append(dado.get('t'))
                    listasy2.append(dado.get('cumu'))
                    listast02.append(dado.get('tc'))
            
            listasX1=[]
            listasY1=[]
            listasT01=[]
            zscore=stats.zscore(np.array(listast01))        
            for zz,xx1,yy1, tt1 in zip(zscore.tolist(),listasx1,listasy1,listast01):
                if abs(zz) < 2.5:
                    listasX1.append(xx1)
                    listasY1.append(yy1)
                    listasT01.append(tt1)
            
            
            listasX2=[]
            listasY2=[]
            listasT02=[]
            zscore=stats.zscore(np.array(listast02))        
            for zz,xx2,yy2, tt2 in zip(zscore.tolist(),listasx2,listasy2,listast02):
                if abs(zz) < 2.5:
                    listasX2.append(xx2)
                    listasY2.append(yy2)
                    listasT02.append(tt2)

            xCrossing=[]
            yCrossing=[]
            for x1, y1  in zip(listasX1, listasY1):
                try:
                    interp1 = scipy.interpolate.InterpolatedUnivariateSpline(x1, y1)
                    for x2, y2 , x0 in zip(listasX2, listasY2, listasT02):
                        try:
                            interp2 = scipy.interpolate.InterpolatedUnivariateSpline(x2, y2)
                            x_at_crossing = scipy.optimize.fsolve(lambda x : np.abs(interp1(x) - interp2(x)), x0=x0)
                            xc=x_at_crossing[0]
                            #yc=interp1(xc)
                            xCrossing.append(xc)
                            #yCrossing.append(yc)
                        except:
                            print("erro amostar", x2)
                except:
                    print("erro amostar", x1)
            zscore=stats.zscore(np.array(xCrossing))        
            for zz,tcc in zip(zscore.tolist(),xCrossing):
                if abs(zz) < 2.5:
                    yCrossing.append(tcc)
            tcmedio.append(np.mean(yCrossing))
            tcerro.append(np.std(yCrossing)/math.sqrt(len(xCrossing)))
        tcnorma=np.array(tcmedio)/tcmedio[0]
        p.plot(concentracao,tcnorma.tolist(),symbol=-1,line=[1,1,''],legend='p')
        


   
    def PlotCruzamentos(self,A=1.7, **kargs):
        concentracao=kargs.get("concentracao")
        tamanhos=kargs.get("tamanhos")
        tcmedio=[]
        tcerro=[]
        p=js.grace()
        p.yaxis(label='tc',charsize=1.50)
        p.xaxis(label='q',charsize=1.50)
        for q in concentracao:
            listasx1=[]
            listasy1=[]
            listast01=[]

            listasx2=[]
            listasy2=[]
            listast02=[]
            for dado in self.dados:
                if dado.get('p')== q and dado.get('L')==tamanhos[0] and dado.get('A')==A:
                    listasx1.append(dado.get('t'))
                    listasy1.append(dado.get('cumu'))
                    listast01.append(dado.get('tc'))
                if dado.get('p')== q and dado.get('L')==tamanhos[1] and dado.get('A')==A:
                    listasx2.append(dado.get('t'))
                    listasy2.append(dado.get('cumu'))
                    listast02.append(dado.get('tc'))
            
            listasX1=[]
            listasY1=[]
            listasT01=[]
            zscore=stats.zscore(np.array(listast01))        
            for zz,xx1,yy1, tt1 in zip(zscore.tolist(),listasx1,listasy1,listast01):
                if abs(zz) < 2.5:
                    listasX1.append(xx1)
                    listasY1.append(yy1)
                    listasT01.append(tt1)
            
            
            listasX2=[]
            listasY2=[]
            listasT02=[]
            zscore=stats.zscore(np.array(listast02))        
            for zz,xx2,yy2, tt2 in zip(zscore.tolist(),listasx2,listasy2,listast02):
                if abs(zz) < 2.5:
                    listasX2.append(xx2)
                    listasY2.append(yy2)
                    listasT02.append(tt2)

            xCrossing=[]
            yCrossing=[]
            for x1, y1  in zip(listasX1, listasY1):
                interp1 = scipy.interpolate.InterpolatedUnivariateSpline(x1, y1)
                for x2, y2 , x0 in zip(listasX2, listasY2, listasT02):
                    interp2 = scipy.interpolate.InterpolatedUnivariateSpline(x2, y2)
                    x_at_crossing = scipy.optimize.fsolve(lambda x : np.abs(interp1(x) - interp2(x)), x0=x0)
                    xc=x_at_crossing[0]
                    #yc=interp1(xc)
                    xCrossing.append(xc)
                    #yCrossing.append(yc)
            zscore=stats.zscore(np.array(xCrossing))        
            for zz,tcc in zip(zscore.tolist(),xCrossing):
                if abs(zz) < 2.5:
                    yCrossing.append(tcc)
            tcmedio.append(np.mean(yCrossing))
            tcerro.append(np.std(yCrossing)/math.sqrt(len(xCrossing)))
        p.plot(concentracao,tcmedio,tcerro,symbol=-1,line=[1,1,''],legend='p')
        tc=[2.0528997, 2.050931718693, 2.048963737387, 2.043059793467, 2.037155849547, 2.031251905627, 2.025347961707, 2.025347961707, 2.021411999094, 2.013540073868, 2.003700167335, 1.997796223415, 1.987956316882, 1.978116410348, 1.964340541202, 1.950564672056, 1.936788802909, 1.917108989843, 1.903333120697, 1.852165606725, 1.844293681498, 1.77935029838]
        p.plot(concentracao,tc,symbol=-1,line=[1,1,''],legend='p')
    
    def crossings(listasX1, listasY1, listasX2, listasY2, x0): # chute inicial 
        self.xCrossing=[]
        self.yCrossing=[]
        for x1, y1  in zip(listasX1, listasY1[i]):
            interp1 = scipy.interpolate.InterpolatedUnivariateSpline(x1, y1)
            for x2, y2  in zip(listasX2, listasY2[i]):
                interp2 = scipy.interpolate.InterpolatedUnivariateSpline(x2, y2)
                x_at_crossing = scipy.optimize.fsolve(lambda x : np.abs(interp1(x) - interp2(x)), x0=x0)
                xc=x_at_crossing[0]
                yc=interp1(xc)
                self.xCrossing.append(xc)
                self.yCrossing.append(yc)
    def cruzamento(self, x1, y1, x2, y2, x0):
                interp1 = scipy.interpolate.InterpolatedUnivariateSpline(x1, y1)
                interp2 = scipy.interpolate.InterpolatedUnivariateSpline(x2, y2)
                x = scipy.optimize.fsolve(lambda x : np.abs(interp1(x) - interp2(x)), x0=x0)
                return x 
     

In [20]:
resultado = Resultado(dados=dados)

In [11]:
resultado.plotDiagrama(L=10, A=1.7, concentracao=concentracao)

In [18]:
resultado.plotDiagramaFiltrado(L=15,A=1.7, concentracao=concentracao)

In [23]:
resultado.plotDiagramaMedio(L=15,A=1.7,concentracao=concentracao)

In [32]:
resultado.plotDiagramaMedio(L=10,A=1.7,concentracao=concentracao)

In [76]:
resultado.plot(L=15,A=1.7, concentracao=concentracao)

In [78]:
resultado.plotMedia(L=15,A=1.7, concentracao=concentracao)

In [79]:
resultado.plotMedia(L=10,A=1.7, concentracao=concentracao)

In [80]:
resultado.plotMedia(L=15,A=1.7,y="calor", concentracao=concentracao)

In [81]:
resultado.plotMedia(L=15,A=1.7,y="cumo", concentracao=concentracao)

In [82]:
resultado.plotMedia(A=1.7,y="cumo", concentracao=concentracao, tamanhos=[10 , 15])

In [143]:
resultado.plotFiltrados(L=15,A=1.65,  concentracao=concentracao)

In [13]:
resultado.plotMedia(A=1.7, concentracao=concentracao, tamanhos=[10 , 15])

In [21]:
resultado.PlotCruzamentosNormal(A=1.7, concentracao=concentracao, tamanhos=[10 , 15])

In [22]:
resultado.PlotCruzamentos(A=1.7, concentracao=concentracao, tamanhos=[10 , 15])

In [14]:
resultado.plotDiagramaCruzamentoMedia(A=1.7, concentracao=concentracao, tamanhos=[10 , 15])

[2.184015995508479, 2.2894890889531148, 2.194876394465573, 1.8956176620538214, 2.2502459963482955, 2.2292735172951765, 2.2429691121778106, 2.251499641266582, 2.2243926546152197, 2.2387434889355107, 2.1413093999001123, 2.2076056878220256, 2.1816258349553417, 2.2015372009942005, 2.1927269052303866, 2.1682769071806796, 1.828992541318682, 1.8387980243546416, 2.0540823544904026, 1.7924425368380008, 2.0445654728480602, 2.0204216093090124]
